In [1]:
#!/usr/bin/env python
from __future__ import unicode_literals, print_function
# import argparse
import ffmpeg
import sys



# parser = argparse.ArgumentParser(description='Generate video thumbnail')
# parser.add_argument('in_filename', help='Input filename')
# parser.add_argument('out_filename', help='Output filename')
# parser.add_argument(
#     '--time', type=int, default=0.1, help='Time offset')
# parser.add_argument(
#     '--width', type=int, default=120,
#     help='Width of output thumbnail (height automatically determined by aspect ratio)')


def generate_thumbnail(in_filename, out_filename, time):

    try:
        (
            ffmpeg
            .input(in_filename, ss=time)
            .filter('scale')
            .output(out_filename, vframes=1)
            .overwrite_output()
            .run(capture_stdout=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        print(e.stderr.decode(), file=sys.stderr)
        sys.exit(1)

In [2]:
steam = ffmpeg.input('mc_start.mp4')

In [3]:
steam.audio.output('a.wav').overwrite_output().run()

(None, None)

In [4]:
probe = ffmpeg.probe('mc_start.mp4')

FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [7]:
def getinfo(in_filename):
    try:
        probe = ffmpeg.probe(in_filename)
    except ffmpeg.Error as e:
        print(e.stderr, file=sys.stderr)
#         sys.exit(1)

    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
#     print(video_stream)
    if video_stream is None:
        print('No video stream found', file=sys.stderr)
#         sys.exit(1)/
    width = int(video_stream['width'])
    height = int(video_stream['height'])
    num_frames = int(video_stream['nb_frames'])
    duration = float(video_stream['duration'])
    
    print('width: {}'.format(width))
    print('height: {}'.format(height))
    print('num_frames: {}'.format(num_frames))
    print('duration: {}'.format(duration))

In [8]:
getinfo('mc_start.mp4')

width: 1280
height: 720
num_frames: 92
duration: 3.68


In [3]:
def read_frame_as_jpeg(in_filename, out_filename,frame_num):
    out, err = (
        ffmpeg
        .input(in_filename)
        .filter('select', 'gte(n,{})'.format(frame_num))
#         .output('pipe:', vframes=1, format='image2', vcodec='mjpeg')
        .output(out_filename, vframes=1).overwrite_output().run(capture_stdout=True)
    )
    return out

In [4]:
def get_pic_frames(in_filename, out_filename, pic_nums):
    probe = ffmpeg.probe(in_filename)
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    num_frames = int(video_stream['nb_frames'])
    
    each_frames = num_frames // pic_nums
    curr_frame = 1
    outs = []
    for i in range(pic_nums):
        read_frame_as_jpeg(in_filename, out_filename + str(i) + '.jpg',curr_frame)
        outs.append(out_filename+str(i)+'.jpg')
        curr_frame += each_frames
        if curr_frame >= num_frames:
            curr_frame = num_frames - 1
        
    return outs

In [5]:
get_pic_frames('mc_start.mp4','rst',5)

['rst0.jpg', 'rst1.jpg', 'rst2.jpg', 'rst3.jpg', 'rst4.jpg']